In [1]:
"""
1) Gain authorization with Google API for sheet
2) Import Sheet(s) as Pandas Dataframe(s)
3) Convert Sheet(s) to CSVs in proper format (note on Monday)
"""
#!pip install -r requirements.txt
#!pip install gspread
#!pip install oauth2client

'\n1) Gain authorization with Google API for sheet\n2) Import Sheet(s) as Pandas Dataframe(s)\n3) Convert Sheet(s) to CSVs in proper format (note on Monday)\n'

In [39]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import fooddatacentral as fdc
import warnings

In [24]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_file = 'eep153-project2-269718.json'
spreadsheet = "16eMvk2KBVOd30v7wTqp1ZGoFno9f9FmfxRBE_jcrLcI"
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
gc = gspread.authorize(credentials)
sh = gc.open_by_key(spreadsheet)

In [25]:
sh.worksheets()

[<Worksheet 'Calories' id:0>,
 <Worksheet 'Food Data' id:1188532609>,
 <Worksheet 'Nutrition_Max' id:3223398>,
 <Worksheet 'Nutrition_Min' id:1079378864>,
 <Worksheet 'CND<>US Conversion' id:1594732978>]

In [26]:
wks_calories = sh.worksheet("Calories")
wks_food_data = sh.worksheet("Food Data")
wks_nutrition_min = sh.worksheet("Nutrition_Min")
wks_nutrition_max = sh.worksheet("Nutrition_Max")

In [27]:
#headers = data.pop(0)
#data_df = pd.DataFrame(data,columns=headers)
# Calories Data
data_calories = wks_calories.get_all_values()
headers = data_calories.pop(0)
df_calories = pd.DataFrame(data_calories,columns=headers)

# Food Data
data_food_data = wks_food_data.get_all_values()
headers = data_food_data.pop(0)
df_food_data = pd.DataFrame(data_food_data,columns=headers)

# Nutrition Data Min
data_nutrition_min = wks_nutrition_min.get_all_values()
headers = data_nutrition_min.pop(0)
df_nutrition_min = pd.DataFrame(data_nutrition_min,columns=headers)

# Nutrition Data Max
data_nutrition_max = wks_nutrition_max.get_all_values()
headers = data_nutrition_max.pop(0)
df_nutrition_max = pd.DataFrame(data_nutrition_max,columns=headers)

In [28]:
#df_calories.head()

In [29]:
df_food_data.head()

,Food,Quantity,Units,Price ($),Date,Location,FDC,Price in $ (100G)
0,Fruit Basket,1,lbs,0.88,1997,USA,,0.19400656
1,Fruit Basket,1,lbs,0.92,1998,USA,,
2,Fruit Basket,1,lbs,1.02,1999,USA,,
3,Fruit Basket,1,lbs,0.97,2000,USA,,
4,Fruit Basket,1,lbs,1,2001,USA,,


In [30]:
df_nutrition_min.head()

,Nutrition,Source of goal,Child 1-3,Female 4-8,Male 4-8,Female 9-13,Male 9-13,Female 14-18,Male 14-18,Female 19-30,Male 19-30,Female 31-50,Male 31-50,Female 51+,Male 51+
0,Source: Appendix 7,Calorie level(s) assessed,"1,000","1,200",1400,"1,600","1,800","1,800",2800,"2,000",2600,"1,800","2,200","1,600","2,000"
1,Protein g,RDA,13,19,19,34,34,46,52,46,56,46,56,46,56
2,Protein % kcal,AMDR,5,10,10,10,10,10,10,10,10,10,10,10,10
3,Carbohydrate g,RDA,130,130,130,130,130,130,130,130,130,130,130,130,130
4,Carbohydrate % kcal,AMDR,45,45,45,45,45,45,45,45,45,45,45,45,45


In [31]:
df_nutrition_max.head()

,Nutrition,Source of goal,Child 1-3,Female 4-8,Male 4-8,Female 9-13,Male 9-13,Female 14-18,Male 14-18,Female 19-30,Male 19-30,Female 31-50,Male 31-50,Female 51+,Male 51+
0,Source: Appendix 7,Calorie level(s) assessed,,,,,,,,,,,,,
1,Protein g,RDA,,,,,,,,,,,,,
2,Protein % kcal,AMDR,,,,,,,,,,,,,
3,Carbohydrate g,RDA,,,,,,,,,,,,,
4,Carbohydrate % kcal,AMDR,65,65,65,65,65,65,65,65,65,65,65,65,65


In [32]:
user = "laialex@berkeley.edu"
apikey = {'laialex@berkeley.edu':"JHdQ6j02nQrZtnwaFLHinmHRPh3J8YAF7rZrzFMY"}

In [33]:
#fdc.search(apikey[user],"crunchy peanut butter")

In [34]:
id = 454532
fdc.nutrients(apikey[user],fdc_id=id)

,Quantity,Units
"Calcium, Ca",62.000,mg
"Iron, Fe",2.250,mg
"Vitamin A, IU",0.000,IU
"Vitamin C, total ascorbic acid",0.000,mg
Riboflavin,0.106,mg
Niacin,12.500,mg
Cholesterol,0.000,mg
Protein,21.880,g
Total lipid (fat),46.880,g
"Carbohydrate, by difference",28.120,g


In [66]:
# Some columns which ought to be numeric are actually str; convert them

# Replace all empty strings with NaN and drop unnecessary columns
df_food_data = df_food_data.replace(['', 'NA'], np.nan)
df_food_data = df_food_data.drop(['Price in $ (100G)'], axis=1, errors='ignore')

# Drop any rows that have any NaN values in crucial columns
df_food_data = df_food_data.dropna(axis=0, how='any', subset=['Date', 'FDC', 'Quantity', 'Price ($)'])

# Convert columns to the appropriate data dtype
df_food_data['Price ($)'] = df_food_data['Price ($)'].astype(float)
df_food_data['Quantity'] = df_food_data['Quantity'].astype(float)
df_food_data['FDC'] = df_food_data['FDC'].astype(int)
df_food_data['Date'] = df_food_data['Date'].astype(int)

df_food_data.head(25)

,Food,Quantity,Units,Price ($),Date,Location,FDC
45,"Flour, white, all purpose",1.0,lbs,0.510000,2008,USA,475388
71,Apples,1.0,lbs,1.320000,2008,USA,592260
113,Butter,1.0,lbs,2.640000,2008,USA,479107
139,Grapefruit,1.0,lbs,0.910000,2008,USA,539389
156,"Sugar, white",1.0,lbs,0.530000,2008,USA,343930
178,Grapes,1.0,lbs,2.210000,2008,USA,173040
203,Lemons,1.0,lbs,1.670000,2008,USA,506187
220,White pan bread,1.0,lbs,1.370000,2008,USA,339504
246,Oranges,1.0,lbs,1.090000,2008,USA,578523
263,Cheddar Cheese,1.0,lbs,4.730000,2008,USA,494681


In [72]:
def get_food_data(df_food_data, year, location):
    """
    Given the general food data dataframe, returns a new food data dataframe filtered to have the 
    correct year and location
    """
    df_result = df_food_data[df_food_data['Date']==year]
    df_result = df_result[df_result['Location']==location]
    return df_result

In [73]:
df_food_data_test = get_food_data(df_food_data, 2008, 'USA')
df_food_data_test.head(25)

,Food,Quantity,Units,Price ($),Date,Location,FDC
45,"Flour, white, all purpose",1.0,lbs,0.510000,2008,USA,475388
71,Apples,1.0,lbs,1.320000,2008,USA,592260
113,Butter,1.0,lbs,2.640000,2008,USA,479107
139,Grapefruit,1.0,lbs,0.910000,2008,USA,539389
156,"Sugar, white",1.0,lbs,0.530000,2008,USA,343930
178,Grapes,1.0,lbs,2.210000,2008,USA,173040
203,Lemons,1.0,lbs,1.670000,2008,USA,506187
220,White pan bread,1.0,lbs,1.370000,2008,USA,339504
246,Oranges,1.0,lbs,1.090000,2008,USA,578523
263,Cheddar Cheese,1.0,lbs,4.730000,2008,USA,494681


In [91]:
def get_nutrients_data(df_food_data):
    """
    Given a food data dataframe, finds the nutritional information for every item under the 'Food' column using 
    the corresponding FDC code and searching the FDC database.
    """
    # N is a mapping of {food: nutritional_data}
    N = {}
    #count = 0
    for food in  df_food_data.Food.tolist():
        try:
            # Retrieve FDC code for this particular code
            FDC_code = df_food_data.loc[df_food_data.Food==food,:].FDC.values[0]
            # Search the FDC database using the FDC Code for nutritional information and add it to N
            N[food] = fdc.nutrients(apikey[user],FDC_code).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC_code))        

    N = pd.DataFrame(N,dtype=float)
    return N

get_nutrients_data(df_food_data_test).head(25)
#df_food_data_test.loc[df_food_data_test.Food=='Apples',:].FDC.values[0]

,"Flour, white, all purpose",Apples,Butter,Grapefruit,"Sugar, white",Grapes,Lemons,White pan bread,Oranges,Cheddar Cheese,...,Whole Milk,Pork (ground),Beef (ground),Fresh Beef,Chicken,Broccoli,Iceberg,Orange Juice,Potatoes,Tomatoes
10:0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.005,NaN,NaN,...,NaN,0.01,0.000,NaN,0.000,0.000,0.000,0.000,0.001,0.000
12:0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.013,NaN,NaN,...,NaN,0.02,0.013,NaN,0.000,0.000,0.000,0.000,0.003,0.000
13:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14:0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.009,NaN,NaN,...,NaN,0.33,0.565,NaN,0.041,0.001,0.000,0.000,0.001,0.000
14:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.016,NaN,NaN,NaN,NaN,NaN
15:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
15:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
16:0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.419,NaN,NaN,...,NaN,4.77,4.104,NaN,1.791,0.029,0.016,0.014,0.016,0.020
16:1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.018,NaN,NaN,...,NaN,0.65,0.689,NaN,0.536,0.000,0.001,0.003,0.001,0.001
17:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.007,NaN,NaN,NaN,NaN,NaN


In [18]:
# Try your own quantities and units.
# If units are missing try adding to ./Data/food_units.txt

print(fdc.units(5,'lbs'))
print(fdc.units(1,'gallon'))
print(fdc.units(2,'tea_bag'))
print(fdc.units(12,'donut'))

22.679618500000004 hectogram
37.85411783999999 deciliter
0.06 hectogram
nan milliliter


In [19]:
# Convert food quantities to FDC units
df_food_Data['FDC Quantity'] = df_food_data[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df_food_data['FDC Price'] = df_food_data['Price']/df_food_data['FDC Quantity']

df_food_data.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df_food_data.groupby('Food')['FDC Price'].min()

Prices.head()

ValueError: ('could not convert string to float: ', 'occurred at index 19')

In [20]:
# Choose sex/age group:
groups = ["Child 1-3", "Female 4-8", "Male 4-8", "Female 9-13", "Male 9-13", "Female 14-18", "Male 14-18", "Female 19-30", "Male 19-30", "Female 31-50", "Male 31-50", "Female 51+", "Male 51+"]
group = "Female 4-8"

# Define *minimums*
bmin = df_nutrition_min.set_index('Nutrition')[group]

# Define *maximums*
bmax = df_nutrition_max.set_index('Nutrition')[group]

# Inspect:
bmin.head()

Nutrition
Source: Appendix 7     1,200
Protein g                 19
Protein % kcal            10
Carbohydrate g           130
Carbohydrate % kcal       45
Name: Female 4-8, dtype: object

In [ ]:
c = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

c

In [ ]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

In [116]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints
# What does concat do???

b

TypeError: bad operand type for unary -: 'str'

In [ ]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(c, -A, -b, method='interior-point')

result

In [ ]:
print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

In [ ]:
# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

In [ ]:
tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
tab

In [ ]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())